# Find proteins outside the 8q loss event (trans) that are commonly different between samples with and without the event

# Only looking at interacting proteins

## Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
CHROMOSOME = "8"
ARM = "q"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_filtered.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,luad_Database_ID,lscc_Database_ID,brca_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
AAR2,NP_001258803.1,NP_001258803.1,NP_001258803.1,NP_001258803,0.100000,0.031243,0.808997,0.278259,0.549109,0.728765,0.257924,0.193667,0.034607,-0.290468,0.177188,-0.107719
ABCF2,NP_005683.2|NP_009120.1,NP_005683.2|NP_009120.1,NP_005683.2|NP_009120.1,NP_005683,0.224826,0.797980,0.462268,0.776484,0.661795,0.945784,0.228875,0.028455,0.102869,0.114274,0.148295,-0.025914
ABL1,NaN,NP_009297.2|NP_005148.2,NP_009297.2|NP_005148.2,NP_005148,0.279869,0.918512,0.530348,0.903833,NaN,0.964206,-0.200430,-0.020740,0.074545,-0.031077,NaN,0.019515
ABLIM1,NP_002304.3|NP_001339371.1|NP_001309821.1|NP_0...,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309815.1,NP_001309811,0.376710,0.663944,0.976421,0.269111,0.609583,0.945784,-0.324995,-0.063420,0.006332,-0.537334,-0.493952,-0.052051
ABLIM1,NP_002304.3|NP_001339371.1|NP_001309821.1|NP_0...,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309815.1,NP_001003407,0.376710,0.663944,0.976421,0.269111,0.609583,0.945784,-0.324995,-0.063420,0.006332,-0.537334,-0.493952,-0.051496
ABLIM1,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309815.1,NP_001309811,0.376710,0.663944,0.976421,0.269111,0.682762,0.945784,-0.324995,-0.063420,0.006332,-0.537334,-0.244554,-0.052051
ABLIM1,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309815.1,NP_001003407,0.376710,0.663944,0.976421,0.269111,0.682762,0.945784,-0.324995,-0.063420,0.006332,-0.537334,-0.244554,-0.051496
ABLIM1,NP_002304.3|NP_001339371.1|NP_001309821.1|NP_0...,NP_001003407.1|NP_002304.3|NP_001309812.1|NP_0...,NP_001309811.1|NP_001309815.1,NP_001309811,0.376710,0.663944,0.976421,0.432840,0.609583,0.945784,-0.324995,-0.063420,0.006332,-0.538375,-0.493952,-0.052051
ABLIM1,NP_002304.3|NP_001339371.1|NP_001309821.1|NP_0...,NP_001003407.1|NP_002304.3|NP_001309812.1|NP_0...,NP_001309811.1|NP_001309815.1,NP_001003407,0.376710,0.663944,0.976421,0.432840,0.609583,0.945784,-0.324995,-0.063420,0.006332,-0.538375,-0.493952,-0.051496


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,AAR2,NP_001258803.1,0.100000,0.257924
1,brca,ABCF2,NP_005683.2|NP_009120.1,0.224826,0.228875
2,brca,ABL1,NP_009297.2|NP_005148.2,0.279869,-0.200430
3,brca,ACACA,NP_942133.1|NP_942134.1|NP_942135.1,0.370913,-0.392848
4,brca,ADSL,NP_000017.1|NP_001304852.1|NP_001116850.1,0.414737,0.192330
5,brca,AGO2,NP_036286.2|NP_001158095.1,0.005572,0.418963
6,brca,AGR2,NP_006399.1,0.010050,-1.770085
7,brca,AJUBA,NP_116265.1|NP_001276026.1,0.796385,-0.108092
8,brca,AKAP5,NP_004848.3,0.067404,-0.899100
9,brca,AKT1,NP_005154.2,0.000919,-0.671610


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)

In [7]:
prots.groupby("cancer_type").count()[["protein"]]

,protein
cancer_type,
brca,89
colon,20
hnscc,1
lscc,3
luad,5


## Find how many cancers each protein was different in

In [8]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
AGO2,"[brca, colon, luad]",3
PTK2,"[brca, colon, luad]",3
FBL,"[brca, colon]",2
HSPE1,"[brca, colon]",2
SMC1A,"[brca, colon]",2
TIMM13,"[brca, colon]",2
MAPKAPK2,"[brca, hnscc]",2
SURF2,"[brca, luad]",2
AGR2,[brca],1


In [9]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

3      2
2      6
1    100
Name: num_cancers, dtype: int64

## Save results

In [10]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}_filtered.tsv"
prots_summary.to_csv(output_file, sep="\t")